In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!pip install tf-nightly

In [4]:
import tensorflow as tf
print(tf.__version__)
tf.test.gpu_device_name()

2.8.0


'/device:GPU:0'

In [4]:
#o = []
#while True:
#  o.append(o)

In [5]:
!ls "/content/drive/MyDrive/ae-test-datasets"

167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0000Z_52.211.21.47_4ue5n8nd.log
167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0240Z_54.154.31.214_38es3f9f.log
167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0250Z_52.211.21.47_4wgzifnz.log
167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0255Z_52.211.21.47_zm3l802v.log
167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0305Z_52.211.21.47_5g2c0cn7.log
167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0310Z_52.211.21.47_59fkgo77.log
167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0310Z_54.154.31.214_7agoebkc.log
167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0320Z_52.211.21.47_3gbz4mur.log
167430

In [6]:
#!pip install --upgrade tensorflow 
#from datasketch import MinHash, MinHashLSH


In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize
#import gensim
#from gensim.models import Word2Vec
import pandas as pd
import re
import numpy as np
from urllib.parse import urlparse,unquote
import time
#import nltk
#nltk.download('punkt')


global vocabulary
vocabulary = {}
vocabulary["_pad_"]=str(0)
vocabulary["_one_char_"]=str(4)
vocabulary["_others_"]=str(2)
vocabulary["_one_dig_"]=str(5)
vocabulary["_eos_"]=str(3)
vocabulary["_sos_"]=str(1)
vocabulary["_num_"]=str(6)
vocabulary["_char_num_"]=str(7)
global vocab_i
global vocfq
vocfq = {}
vocfq["_pad_"]=0
vocfq["_one_char_"]=0
vocfq["_others_"]=0
vocfq["_one_dig_"]=0
vocfq["_eos_"]=0
vocfq["_sos_"]=0
vocfq["_num_"]=0
vocfq["_char_num_"]=0
vocab_i = 8
stringhashes = {}
traininginputs = {}
ddirectories = {}
dqueries = {}
trainees = []


In [6]:
#http logs for offline training
def getinputfile(file):
    rev = pd.read_csv(file, delim_whitespace=True,header=None)
    return rev

#get the related part from logs DF
def processfile(data):
    processed = []
    for i,_ in enumerate(data):
        #replaced = re.sub(",|\||%|/|\.|\?|&|-|=|\+|:|_|}|{|!|\$|'|\(|\)|\*"," ",_)
        #replaced = ' '.join(e for e in _ if e.isalnum())
        #print(_)
        replaced = str(_)
        replaced = unquote(replaced)
        replaced = re.sub('[^A-Za-z0-9]+', ' ', replaced)
        replaced = re.sub("(?<!\S)\d(?!\S)", " _one_dig_ ",replaced)
        replaced = re.sub("(^|\s+)\w(?=\s+|$)", " _one_char_ ", replaced)  
        replaced = re.sub("\\b(\\d+)\\b", " _num_ ", replaced)
        #replaced = re.sub("\\b([a-zA-Z]\\d+)\\b", " _char_num_ ",replaced)
        #print(replaced)
        processed.append("_sos_ "+replaced + " _eos_"+" _pad_")
    return processed

#parse payload
def parsepayload(payload):
    
    return

trainingdata = {}
#tokenize training corpus
def tokenizetraining(data):
    data = pd.Series(data)
    tok = []
    for i,_ in enumerate(data):
        #print(_)
        while unquote(_) != _:
            _ = unquote(_) 
        x = tokenizepayload(_)
        rev = mapt2int(x,vocabulary)
              
        if rev not in tok:
            tok.append(rev)
        
    return tok

#def tokenizetraining(data):
#    tok = []
#    for i, _ in enumerate(data):
#        res=[]
#        print(_)
#        rev = _.split()
#        for x in rev:
#            res.append(c.wv.word_vec(x.lower()))
#            
#        tok.append(res)
#    return tok
#prepare corpus
def preparecorpus(rev):
    processed = processfile(rev[:])
    #proc = rev[:100000][5]
    corpus_text = ' '.join(processed)
    data = []
    # iterate through each sentence in the file
    for i in sent_tokenize(corpus_text):
        temp = []
        # tokenize the sentence into words
        for j in word_tokenize(i):
            temp.append(j.lower())
        data.append(temp)
        #data=processfile(data)
    return data, processed

def preparevocabulary(corpus):
    global vocabulary, vocab_i
    for _ in corpus:
        #for _x in _:
        if _ in vocabulary:
            vocfq[_] += 1
            continue
        else:
            vocabulary[_] = str(vocab_i)
            vocfq[_]=1
            vocab_i += 1
    return


#embedding to create vocabulary
#def embedwords(parsedpayload):
#    model = Word2Vec(parsedpayload, size=5,min_count = 1, window = 5, sg=0,workers=8) 
#    return model

#tokenize the newly arrived http request
def tokenizepayload(payload):
    #print(payload)
    payload=str(payload)
    payload = payload.lower()
    replaced = unquote(payload)
    replaced = re.sub('[^A-Za-z0-9]+', ' ', replaced)
    replaced = re.sub("(?<!\S)\d(?!\S)", " _one_dig_ ",replaced)
    replaced = re.sub("(^|\s+)\w(?=\s+|$)", " _one_char_ ", replaced) 
    replaced = re.sub("\\b(\\d+)\\b", " _num_ ", replaced)
    #replaced = re.sub("\\b([a-zA-Z]\\d+)\\b", " _char_num_ ",replaced)
    splitted = replaced.split()
    if len(splitted)>490:
        replaced = " ".join(splitted[:250])
    replaced = "_sos_ "+ replaced + " _eos_"
    #print(replaced)
    #replaced = "".join(replaced)
    
    #print(replaced)
    return replaced

#def processpayload(data,w2vmodel):
#    processed = []
#    for i,_ in enumerate(data):
        #replaced = re.sub(",|\||%|/|\.|\?|&|-|=|\+|:|_|}|{|!|\$|'|\(|\)|\*"," ",_)
        #replaced = ' '.join(e for e in _ if e.isalnum())
        #print(_)
#        replaced = unquote(_)
#        replaced = re.sub('[^A-Za-z0-9]+', ' ', replaced)
#        replaced = re.sub("(?<!\S)\d(?!\S)", " _one_dig_ ",replaced)
#        replaced = re.sub("(^|\s+)\w(?=\s+|$)", " _one_char_ ", replaced)
#        replaced = re.sub("\\b(\\d+)\\b", " _num_ ", replaced)
        #replaced = re.sub("\\b([a-zA-Z]\\d+)\\b", " _char_num_ ",replaced)
        #print(replaced)
#        p = "_sos_ "+replaced + " _eos_"
#        p=p.lower().split()
#        p_count = 80-len(p)
#        p=p+["_pad_"]*p_count
#        pp = [(w2vmodel.wv.get_vector(x)).tolist() for x in p]
#        processed.append(np.asarray(pp).reshape(80,5))
        
#    return processed  

#embed online inputs
def mapt2int(data,vocab):
    #print(data)
    #data = tokenizepayload(data)
    #print(data)
    #for key, value in vocab.items():
        #print(key)
    #print("Data : "+str(data))
    #dd=[]
    for word in data.split():
        
        if word in vocab:
            data = data.replace(word, str(" "+vocab[word])+" ",1)
        else: 
            data = data.replace(word, " 2 ",1)
    return data

def prepareforml(data):
    mldata = []

    if len(data)==1:
        mldata.append(data[0].split())
        return mldata
    else:
        for _ in data:
            mldata.append(_.split())
        return mldata

def preparetraininoutput(data):
    preprocessed =[]
    aa = [0]*len(vocabulary)
    for _ in data:
        processed =[]
        #print(_)
        for x in _.split():
            dd = aa
            dd[int(x)]=1
            processed.append(dd)
        preprocessed.append(processed)
    return preprocessed

def parseurls(a):
    if not isinstance(a, pd.Series):
        a=pd.Series(a)    
    headers= []
    for _ in a:
        url = _.split()[1]
        #while "%" in url or not stopiter:
        while unquote(url) != url:
            url = unquote(url)            
        headers.append(url_parser(url))
    return headers


def url_parser(url):
    
    parts = urlparse(url)
    directories = parts.path.strip('/').split('/')
    unamp = re.sub("&amp;", "&" ,parts.query)
    queries = unamp.strip('&').split('&')
    queriesdict = dict((s.split('=')+[1])[:2] for s in queries)
    
    elements = {
        'scheme': parts.scheme,
        'netloc': parts.netloc,
        'path': parts.path,
        'params': parts.params,
        'query': parts.query,
        'fragment': parts.fragment,
        'directories': directories,
        'queries': queries,
        'queriesdict': queriesdict,
    }
    
    return elements

traininginputs = {}
#def preparefortraining(data):
#    print(len(data))
#    ax = []
#    i=0
#    for _ in data:
#        trainingarr =[]
#        parsed = tokenizepayload(_)
#        splitted = parsed.split()
        #splitted_len = len(splitted)
        #print(splitted_len)
        # if splitted_len not in traininginputs:
        #     trainingarr.append(_)
        #     traininginputs[splitted_len]=trainingarr
        #     ax.append(_)
        # elif splitted_len in traininginputs:
        #     if _ in traininginputs[splitted_len]:
        #         continue
        #     elif _ not in traininginputs[splitted_len]:
        #         trainingarr.append(_)
        #         traininginputs[splitted_len].append(trainingarr)
        #         ax.append(_)
        #for c, i in enumerate(data):
        #    t = time.time()
#        minhash = MinHash(num_perm=64)
#        for d in tokenizepayload(_).split():
            #print(d)
#            minhash.update(d.encode('utf-8'))
            #print(c)
#        result = lsh.query(minhash)
        #print(_)
        #print(result)
#        if len(result)==0:
            #print("no entry")
#            lsh.insert(i, minhash)
#            traininginputs[i]=minhash
#            i +=1
#            ax.append(_)
#        if len(result)>0:
            #print("entry")
#            ax.append(_)
#    print(i)
#    return pd.Series(ax)
def preparecorpusandvocabulary(headers):
    corpus=set()
    for _ in headers:
        #print(_)
        # for x in _["scheme"]:
        #     y = parsewords(x)
        #     for x in y.split():
        #print(_["scheme"])
        corpus.add(_["scheme"])
        #for x in _["netloc"]:
        y = parsewords(_["netloc"])
        for x in y.split():
            corpus.add(x)
        for x in _["directories"]:
            y = parsewords(x)
            for x in y.split():
                corpus.add(x)
        for x in _["queries"]:
            #print(x)
            #print(x.partition("=")[0])
            y = parsewords(x.partition("=")[0])
            for z in y.split():
                corpus.add(y)
    preparevocabulary(corpus)
    return corpus

#tokenize the newly arrived http request
def parsewords(payload):
    #print(payload)
    payload=str(payload)
    payload = payload.lower()
    #replaced = unquote(payload)
    replaced = re.sub('[^A-Za-z0-9]+', ' ', payload)
    replaced = re.sub("(?<!\S)\d(?!\S)", " _one_dig_ ",replaced)
    replaced = re.sub("(^|\s+)\w(?=\s+|$)", " _one_char_ ", replaced) 
    replaced = re.sub("\\b(\\d+)\\b", " _num_ ", replaced)
    #replaced = re.sub("\\b([a-zA-Z]\\d+)\\b", " _char_num_ ",replaced)
    #print(replaced)
    #replaced = "".join(replaced)
    
    #print(replaced)
    return replaced


In [7]:
import os
from os import listdir
from os.path import isfile, join

for root, dirs, files in os.walk("/content/drive/MyDrive/ae-test-datasets"):
   path = root.split(os.sep)
   for index, file in enumerate(files):
      im2 = [ f for f in listdir(root) if isfile(join(root,f)) ]

In [10]:
#a1 = getinputfile("/content/drive/MyDrive/167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0000Z_52.211.21.47_4ue5n8nd.log")
#a1 = a1[:][12]
#a2 = getinputfile("/content/drive/MyDrive/167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0400Z_52.211.21.47_4qfgr38n.log")
#a2 = a2[:][12]
#a3 = getinputfile("/content/drive/MyDrive/167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0530Z_52.211.21.47_1aiwi3xk.log")
#a3 = a3[:][12]

In [11]:
a = pd.Series()
for _ in im2[:]:
  x = getinputfile("/content/drive/MyDrive/ae-test-datasets/"+_)
  a = pd.concat([a,x[:][12]],ignore_index=True, sort=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [12]:
trainees=[]

In [13]:
def create_dict(l, d):
    if len(l) == 1:
        #print("L :"+ str(l)+ "..."+str(type(l)))
        #d=dict(d)
        d[l[0]] = {}
    else:
        #print("D : "+str(d) +"...")
        #d = dict(d)
        if l[0] == "customers":
            l[1] = "c_id"
        if l[0] == "products":
            l[1] = "p_id"
        if l[0] == "product-alternative-colors":
            l[1] = "p_color_id"
        #d = d.setdefault(l[0], {})
        d[l[0]]={}
        create_dict(l[1:], d[l[0]])

def check_existence(k, d):
    # we will use a way to get a dictionary about url parts like /customers or /products
    if len(k)==1:
        if k[0] in d:
            if d[k[0]] == {}:
                return True
            else: 
                return False
        else: 
            return False
    else:
        if k[0] not in d:
            return False
        else:
            if k[0] == "customers":
                k[1] = "c_id"
            if k[0] == "products":
                k[1] = "p_id"
            if k[0] == "product-alternative-colors":
                k[1] = "p_color_id"
            return check_existence(k[1:], d[k[0]])

def createqdict(k):
    exist = False
    for _qkey in k:
        if _qkey != "":
            if "|" in str(k[_qkey]):
                k[_qkey] = k[_qkey].split("|")
            if not isinstance(k[_qkey],list):
                k[_qkey] = [k[_qkey]]
            #if  isinstance(ss["queriesdict"][_qkey], int):
            #ss["queriesdict"][str(_qkey)]=set([ss["queriesdict"][_qkey]])
            if not str(_qkey) in dqueries:
                exist = False
                dqueries[str(_qkey)]=set()
                dqueries[str(_qkey)].update(set(k[_qkey]))
            else:
                if set(k[_qkey]).difference(dqueries[str(_qkey)]) != set():
                    exist = False
                else:
                    exist =True
                dqueries[str(_qkey)].update(set(k[_qkey]))
    return exist
    
t = time.time()
for _ in a:
    ss =unquote(_.split()[1])
    while unquote(ss) != ss:
        ss = unquote(ss)
    ss = url_parser(ss)
    #_dict = list_to_dict(ss["directories"])
    #print(ss["directories"])
    ex = check_existence(ss["directories"], ddirectories)
    if not ex:
        #trainees.append(_)
        create_dict(ss["directories"],ddirectories)  
    qex = createqdict(ss["queriesdict"])
    if not qex and not ex:
        trainees.append(_)
    # for _qkey in ss["queriesdict"]:
    #     if _qkey != "":
    #         if "|" in str(ss["queriesdict"][_qkey]):
    #             ss["queriesdict"][_qkey] = ss["queriesdict"][_qkey].split("|")
    #         if not isinstance(ss["queriesdict"][_qkey],list):
    #             ss["queriesdict"][_qkey] = [ss["queriesdict"][_qkey]]
    #         #if  isinstance(ss["queriesdict"][_qkey], int):
    #         #ss["queriesdict"][str(_qkey)]=set([ss["queriesdict"][_qkey]])
    #         if not str(_qkey) in dqueries:
    #             dqueries[str(_qkey)]=set()
    #             dqueries[str(_qkey)].update(set(ss["queriesdict"][_qkey]))
    #         else:
    #             dqueries[str(_qkey)].update(set(ss["queriesdict"][_qkey]))
           
print(time.time()-t)
print((time.time()-t)/len(a))

343.60554790496826
0.0003825817675352894


In [14]:
print(len(a))
print(len(trainees))

898124
194111


In [15]:
#a = preparefortraining(a)

In [8]:
from tensorflow.keras.layers import Dense, Input,LSTM, TimeDistributed,RepeatVector
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
MAX_LEN = 256

In [10]:
def decoder_output_creater(decoder_input_data, num_samples, MAX_LEN, VOCAB_SIZE):
  
  decoder_output_data = np.zeros((num_samples, MAX_LEN, VOCAB_SIZE), dtype="float32")

  for i, seqs in enumerate(decoder_input_data):
      for j, seq in enumerate(seqs):
          if j > 0:
              decoder_output_data[i][j][seq] = 1.
  #print(decoder_output_data.shape)
  
  return decoder_output_data

In [19]:
#a = pd.concat([a1,a2,a3],ignore_index=True, sort=False)
b = parseurls(a)

#MAX_LEN = max([len(_.split()) for _ in f])

In [20]:
c = preparecorpusandvocabulary(b)


In [88]:
reverse_vocab = dict((i, word) for word, i in vocabulary.items())

In [21]:
 #zzz = tokenizetraining(trainees)

In [22]:
#from tensorflow.keras.optimizers import Adam
#ADAM = Adam(lr=0.05, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.5, amsgrad=False, clipnorm=1.)


In [11]:
def input_generator(batchsize,adata):
   i=0
   inputs = []
   targets = []
   while True:
       f = tokenizetraining(adata[i*batchsize:(i+1)*batchsize]) 
       g = prepareforml(f)
       padded_docs = pad_sequences(g, maxlen=MAX_LEN, padding='post')
       dec_out = decoder_output_creater(padded_docs, len(g), MAX_LEN, len(vocabulary))
       i +=1
       yield padded_docs, dec_out
       
       i = 0

In [12]:
#b,d = preparecorpus(a)
#e = preparevocabulary(b)


#c = embedwords(b)
#c.train(processfile(a[:100000][5]), total_examples=c.corpus_count, epochs=5)
#e = preparevocabulary(b)
#f = pd.Series(f)
#MAX_LEN = 935
# import time
# t = time.time()
#f = tokenizetraining(a[:1500000])
# print(time.time()-t)

# t = time.time()
# h = preparetraininoutput(f)
# print(time.time()-t)
# #y= prepareforml(h)
# #f = [[c.wv.getc_] for _ in processfile(a[:100000][5])]
# #from gensim.models import KeyedVectors


def encoder_decoder_model_with_attension(
        encoder_vocab_size,
        decoder_vocab_size,
        max_encoder_seq_length,
        max_decoder_seq_length):
    #  define length of encoder/decoder input
    encoder_input = Input(shape=(max_encoder_seq_length, ))
    #decoder_input = Input(shape=(max_decoder_seq_length, ))
    # Encoder
    # we can add BatchNormalization, Masking
    encoder_input2embedding = Embedding(input_dim=encoder_vocab_size, output_dim=64, input_length=max_encoder_seq_length, mask_zero=True)(encoder_input)
    embedding2encoder = LSTM(units=256, return_sequences=True, dropout=0.3)(encoder_input2embedding)
    embedding2encoder2 = LSTM(units=128, return_sequences=False, dropout=0.3)(embedding2encoder)
    repeat2encoder = RepeatVector(MAX_LEN)(embedding2encoder2)
    # Decoder
    # we can add BatchNormalization, Masking
    decoderlstm2 = LSTM(units=128, return_sequences=True, dropout=0.3)(repeat2encoder)
    decoderlstm = LSTM(units=256, return_sequences=True, dropout=0.3)(decoderlstm2)
    context2softmax = Dense(units=decoder_vocab_size, activation="softmax")
    softmax2output = TimeDistributed(context2softmax)(decoderlstm)
    # Attention Encoder-Decoder Model
    model = Model(inputs=[encoder_input], outputs=[softmax2output])
    return model

model = encoder_decoder_model_with_attension(len(vocabulary), len(vocabulary), MAX_LEN, MAX_LEN)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=["acc"])
#model.compile(optimizer=ADAM, loss="categorical_crossentropy",metrics=["acc"])

#from keras.utils.vis_utils import plot_model
#plot_model(model)



from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

def data_spliter(encoder_input_data, decoder_input_data, test_size1=0.2, test_size2=0.3):
  
  en_train, en_test, de_train, de_test = train_test_split(encoder_input_data, decoder_input_data, test_size=test_size1)
  en_train, en_val, de_train, de_val = train_test_split(en_train, de_train, test_size=test_size2)
  
  yield en_train, en_val, en_test, de_train, de_val, de_test

#en_train, en_val, en_test, de_train, de_val, de_test = data_spliter(padded_docs, dec_out)


In [25]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256)]             0         
                                                                 
 embedding (Embedding)       (None, 256, 64)           820288    
                                                                 
 lstm (LSTM)                 (None, 256, 256)          328704    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 repeat_vector (RepeatVector  (None, 256, 128)         0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 256, 128)          131584    
                                                             

In [26]:
model.fit(input_generator(192,trainees), steps_per_epoch=(len(trainees) / 192),epochs=4)


Epoch 1/4
1010/1010 [==============================] - 1301s 1s/step - loss: 0.2744 - acc: 0.9379
Epoch 2/4
1010/1010 [==============================] - 1282s 1s/step - loss: 0.0632 - acc: 0.9737
Epoch 3/4
1010/1010 [==============================] - 1279s 1s/step - loss: 0.0315 - acc: 0.9859
Epoch 4/4
1010/1010 [==============================] - 1284s 1s/step - loss: 0.0151 - acc: 0.9918


In [27]:
tf.keras.models.save_model(model,'/content/drive/MyDrive/ae220302.h5',overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
) 

In [29]:
model.save('/content/drive/MyDrive/ae220302-x.h5')
model.save_weights('/content/drive/MyDrive/ae220302-x-weights.h5')

In [13]:
import gc
gc.collect()

44

In [14]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/ae220302.h5')
keras.backend.set_learning_phase(0)

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [81]:
datasettotest = getinputfile("/content/drive/MyDrive/ae-test-datasets/167430609062_elasticloadbalancing_eu-west-1_app.modanisa-external.c514a951a6e0bfb3_20220101T0755Z_52.211.21.47_1ff95gnt.log")
datasettotest = datasettotest[12]
#parsetest = parseurls(datasettotest)

In [ ]:
parsetest

In [ ]:
datasettotest[20]

'GET https://fcdn.modanisa.com:443/r/pro2/2020/11/06/u-desenli-kemer-detayli-kaban--siyah-mor--tesse-8073104-2.jpg HTTP/2.0'

In [ ]:
f = tokenizetraining(datasettotest[0]) 
g = prepareforml(f)

In [ ]:
padded_docs = pad_sequences(g, maxlen=MAX_LEN, padding='post')
dec_out = decoder_output_creater(padded_docs, len(g), MAX_LEN, len(vocabulary))

In [93]:
t= time.time()
sent = []
inp_sent = []
toks= []
rectoks=[]
for _ in attacklist:
  tokened = tokenizetraining(_)
  toks.append(tokened)
  toml = prepareforml(tokened)
  padded = pad_sequences(toml, maxlen=MAX_LEN, padding='post')
  pred = model.predict(padded)
  sentence = ""
  recs = []
  for i in range(len(pred[0])):
    y = np.argmax(pred[0][i])
    recs.append(y)
    try:
      sentence+=(" "+reverse_vocab[str(y)])
    except:
      sentence += (" "+"_others_")   
  rectoks.append(recs) 
  sent.append(sentence)
  inp_sentence = ""
  for _ in tokened[0].split():
    try:
      inp_sentence +=(" "+reverse_vocab[str(_)])
    except:
      inp_sentence += (" "+"_others_")
  inp_sent.append(inp_sentence)
print(time.time()-t)

75.18403482437134


In [98]:
xxx= pd.DataFrame([toks,rectoks]).T

In [99]:
xxx

,0,1
0,[ 1 5712 10352 2 10559 12729 8342 ...,"[5712, 5712, 10352, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,[ 1 5712 10352 2 10559 12729 4352 ...,"[12364, 12364, 10352, 5659, 10559, 12729, 6, 1..."
2,[ 1 5712 10352 2 10559 12729 6623 ...,"[5712, 5712, 10352, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,[ 1 5712 10352 2 10559 12729 4352 ...,"[12364, 12364, 10352, 5659, 10559, 12729, 6, 1..."
4,[ 1 5712 10352 2 10559 12729 4352 ...,"[12364, 12364, 10352, 5659, 10559, 12729, 6, 1..."
...,...,...
716,[ 1 5712 10352 2 10559 12729 4352 ...,"[12364, 5712, 10352, 5659, 10559, 12729, 6, 10..."
717,[ 1 5712 10352 2 10559 12729 4352 ...,"[12364, 5712, 10352, 5659, 10559, 12729, 6, 10..."
718,[ 1 5712 10352 2 10559 12729 4352 ...,"[12364, 5712, 10352, 5659, 10559, 12729, 6, 10..."
719,[ 1 5712 10352 2 10559 12729 4352 ...,"[12364, 5712, 10352, 5659, 10559, 12729, 6, 10..."


In [76]:
import nltk

In [94]:
s1 = [re.sub("_pad_","",_).split() for _ in sent]
s2 = [re.sub("_pad_","",_).split() for _ in inp_sent]


In [95]:
scores=[]
inp = []
rec=[]
score = []
for i in range(len(s1)):
  s=nltk.translate.bleu_score.sentence_bleu([s1[i]], s2[i])
  scores.append(s)
  if s < 0.5:
    inp.append(s1[i])
    rec.append(s2[i])
    score.append(s)
import pandas as pd
xx = pd.DataFrame([inp,rec,score]).T


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [96]:
xx



,0,1,2
0,"[get, get, https]","[_sos_, get, https, _others_, modanisa, com, e...",0.408248
1,"[delete, delete, https, api1, modanisa, com, _...","[_sos_, get, https, _others_, modanisa, com, c...",0.385386
2,"[get, get, https]","[_sos_, get, https, _others_, modanisa, com, a...",0.408248
3,"[delete, delete, https, api1, modanisa, com, _...","[_sos_, get, https, _others_, modanisa, com, c...",0.385386
4,"[delete, delete, https, api1, modanisa, com, _...","[_sos_, get, https, _others_, modanisa, com, c...",0.385386
...,...,...,...
595,"[delete, get, https, api1, modanisa, com, _num...","[_sos_, get, https, _others_, modanisa, com, c...",0.438113
596,"[delete, get, https, api1, modanisa, com, _num...","[_sos_, get, https, _others_, modanisa, com, c...",0.438113
597,"[delete, get, https, api1, modanisa, com, _num...","[_sos_, get, https, _others_, modanisa, com, c...",0.438113
598,"[delete, get, https, api1, modanisa, com, _num...","[_sos_, get, https, _others_, modanisa, com, c...",0.452985


In [ ]:
global vocabulary
vocabulary = {}
vocabulary["_pad_"]=str(0)
vocabulary["_one_char_"]=str(4)
vocabulary["_others_"]=str(2)
vocabulary["_one_dig_"]=str(5)
vocabulary["_eos_"]=str(3)
vocabulary["_sos_"]=str(1)
vocabulary["_num_"]=str(6)
vocabulary["_char_num_"]=str(7)
global vocab_i
global vocfq
vocfq = {}
vocfq["_pad_"]=0
vocfq["_one_char_"]=0
vocfq["_others_"]=0
vocfq["_one_dig_"]=0
vocfq["_eos_"]=0
vocfq["_sos_"]=0
vocfq["_num_"]=0
vocfq["_char_num_"]=0
vocab_i = 8
stringhashes = {}
traininginputs = {}
ddirectories = {}
dqueries = {}
trainees = []


In [15]:
import pickle


In [59]:
varsss = [vocabulary,vocfq,vocab_i,vocfq,dqueries,ddirectories]
varsstr = ["vocabulary","vocfq","vocab_i","dqueries","ddirectories"]

for i in range(len(varsss)):
  with open('/content/drive/MyDrive/variables'+str(varsstr[i])+'.pickle', 'wb') as f:
    pickle.dump(varsss[i],f)

In [16]:
with open('/content/drive/MyDrive/variablesvocabulary.pickle', 'rb') as f:
   vocabulary =pickle.load(f)
with open('/content/drive/MyDrive/variablesvocfq.pickle', 'rb') as f:
   vocfq =pickle.load(f)
with open('/content/drive/MyDrive/variablesvocab_i.pickle', 'rb') as f:
   vocab_i =pickle.load(f)

with open('/content/drive/MyDrive/variablesdqueries.pickle', 'rb') as f:
   dqueries =pickle.load(f)
with open('/content/drive/MyDrive/variablesddirectories.pickle', 'rb') as f:
   ddirectories =pickle.load(f)

In [76]:
import csv
atacktest = pd.read_csv("/content/drive/MyDrive/ae-test-datasets/RAW2.csv")

ParserError: ignored

In [17]:
### Loop the data lines
with open("/content/drive/MyDrive/ae-test-datasets/RAW2.csv", 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]

### Read csv
df = pd.read_csv("/content/drive/MyDrive/ae-test-datasets/RAW2.csv", header=None, delimiter=",", names=column_names)


In [22]:
dfcsv= df

In [34]:
import csv
result = [row for row in csv.reader(dfcsv.splitlines(), delimiter=';')]

AttributeError: ignored

In [41]:
dflist=dfcsv.to_list()

In [73]:
attacklist = []
for _ in dflist:
  x = _[0][1:-1] +" "+_[1][1:-1]+"://"+_[2][1:-1]+_[3][1:-1]+"?"+_[4][1:-1]
  attacklist.append(x)

In [97]:
len(attacklist)

721